In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Parameters
samplenum = 1
input_size = 3
output_size = 1
time_length = 30; #seconds

In [3]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)
print(state_init.y)
print(dy.shape)
#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
print(y_target)
p = dyn.get_p(y_target.transpose(), dyn.p_init)
y_target= torch.tensor(y_target, requires_grad= True)
print(p.dtype)

[2.15365312e-05 1.99969848e+00 0.00000000e+00 2.15757288e-05
 1.99969848e+00 0.00000000e+00 2.15562034e-05 1.99969848e+00
 0.00000000e+00 2.14781676e-05 1.99969848e+00 0.00000000e+00
 2.13419924e-05 1.99969848e+00 0.00000000e+00 2.11482057e-05
 1.99969848e+00 0.00000000e+00 2.08974905e-05 1.99969848e+00
 0.00000000e+00 2.05906827e-05 1.99969848e+00 0.00000000e+00
 2.02287680e-05 1.99969848e+00 0.00000000e+00 1.98128796e-05
 1.99969848e+00 0.00000000e+00 1.93442938e-05 1.99969848e+00
 0.00000000e+00 1.88244269e-05 1.99969848e+00 0.00000000e+00
 1.82548306e-05 1.99969848e+00 0.00000000e+00 1.76371875e-05
 1.99969848e+00 0.00000000e+00 1.69733060e-05 1.99969848e+00
 0.00000000e+00 1.62651155e-05 1.99969848e+00 0.00000000e+00
 1.55146602e-05 1.99969848e+00 0.00000000e+00 1.47240939e-05
 1.99969848e+00 0.00000000e+00 1.38956733e-05 1.99969848e+00
 0.00000000e+00 1.30317519e-05 1.99969848e+00 0.00000000e+00
 1.21347732e-05 1.99969848e+00 0.00000000e+00 1.12072639e-05
 1.99969848e+00 0.000000

## Building the custon Simulation Activation Function

In [5]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        return grad_input
Simulate = Simulate.apply

class Lossfunction(nn.Module):

    def __init__(self, n_in, out_sz):
        super(Lossfunction, self).__init__()

        self.criterion = nn.MSELoss()
    
    def forward(self, input):
        p0 = dyn.p_init
        y = y_target.clone()
        y_pred = Simulate(input)
        e1 = (input[0:3*(30-1)] - input[3:3*30]).pow(2).sum()
        e2 = self.criterion(input[0:3].double(), torch.tensor(p0[0:3]))
        e3 = self.criterion(y_pred.float(), y)
        Loss = e1 + e2 + e3
        return Loss
    
model = Lossfunction(3*30, output_size) # Input size is now size of p and not size of y_target



In [11]:
from torch.autograd import gradcheck

#p = dyn.p_init
p = torch.tensor(p, requires_grad = True)
input = (p.double())
#print(input)
test = gradcheck(model, (input,), eps=1e-5, atol=1e-4, raise_exception = True)
print(test)

True
